In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import string
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer,CountVectorizer
from nltk.stem.porter import PorterStemmer
dataset=pd.read_csv(r'C:\Users\HP\Desktop\ML\Reviews.csv')
dataset=dataset.drop_duplicates(subset={'UserId','Time'},keep='first',inplace=False)
dataset=dataset.sample(20000)
dataset.reset_index(inplace = True) 
print(len(dataset))
dataset=dataset.sort_values('Time',ascending=True)

20000


In [2]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer,SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
import nltk
#nltk.download('wordnet')
  
#lemmatizer = WordNetLemmatizer()

stop=dict.fromkeys(stopwords.words('english'),1)
sno=SnowballStemmer('english')

def clean_html(sentence):
    x=re.compile('<.*?>')
    cleantext=re.sub(x,' ',sentence)
    return cleantext

def clean_punc(sentence):
    cleaned=re.sub(r'[?|!|\'|"|#|.|,|)|(|\|/]',r' ',sentence)
    return cleaned

p_text=[] #processed text
from tqdm import tqdm
for sent in tqdm(dataset['Text'].values,position=0,leave=True):
    sent=clean_html(sent)
    sent=clean_punc(sent)
    s=[]
    for w in sent.split():
        if(w.isalpha() and len(w)>2 and w.lower() not in stop):
            w=(sno.stem(w.lower()))
            #w=lemmatizer.lemmatize(w.lower()) stemming was producing better results
            s.append(w)
    sent=" ".join(s)
    p_text.append(sent)
    
dataset['p_text']=p_text

dataset['p_text'][377]

100%|███████████████████████████████████████████████████████████████████████████| 20000/20000 [00:29<00:00, 670.70it/s]


'first got item time complain lollipop tasti sort sour sour delici wait tri lollipop brand'

In [4]:
from gensim.models import Word2Vec
sentences=[]
for sent in tqdm(p_text,position=0,leave=True):
    sentences.append(sent.split())

model=Word2Vec(sentences,min_count=2,size=200,workers=16)
w2v_words=dict(model.wv.vocab)
len(w2v_words)

100%|█████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 80359.12it/s]


10502

In [5]:
text_vectors=[]
            
for sent in tqdm(sentences,position=0,leave=True):
    text_vec=np.zeros(200)
    word_cnt=0
    for word in sent:
        if word in w2v_words:
            text_vec+=model.wv[word]
            word_cnt+=1
            
    if word_cnt!=0:
        text_vec/=word_cnt
    text_vectors.append(text_vec)

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:07<00:00, 2662.89it/s]


In [6]:
X=text_vectors
Y=[]
p_reviews=0
n_reviews=0
for score in dataset['Score'].values:
    if score >=4:
        Y.append(1)
        p_reviews+=1
    else:
        Y.append(0)
        n_reviews+=1
        
dataset['class']=Y
print(len(Y))
print(p_reviews)

20000
15484


In [52]:
from sklearn.model_selection import train_test_split

tuned_params=[{'C':[10**4,10**2,10**0,10**-2,10**-4]}]
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.3,random_state=30)

#standardization
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train= scaler.fit_transform(x_train)
print(x_train.mean())
print(x_train.std())

8.196617987518299e-19
1.0000000000000002


In [53]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

model=GridSearchCV(LogisticRegression(class_weight={1:1,0:2},solver='lbfgs',n_jobs=-1),tuned_params,scoring='accuracy',cv=5,n_jobs=-1)
model.fit(x_train,y_train)

print(model.best_estimator_)
print(model.score(x_test,y_test))


LogisticRegression(C=0.0001, class_weight={1: 1, 0: 2}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=-1, penalty='l2', random_state=None,
          solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)
0.8105


In [59]:
lrmodel=LogisticRegression(class_weight={1:1,0:2},solver='lbfgs',penalty='l2',C=.1,n_jobs=-1,max_iter=1000)
lrmodel.fit(x_train,y_train)

w=lrmodel.coef_
print(np.count_nonzero(w))

200


In [63]:
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix

x_test=scaler.transform(x_test)
pred=lrmodel.predict(x_test)
test_a=accuracy_score(y_test,pred)*100
test_p=precision_score(y_test,pred)
test_r=recall_score(y_test,pred)
test_f=f1_score(y_test,pred)
test_c=confusion_matrix(y_test,pred)
tn, fp, fn, tp=test_c.ravel()
print("Accuracy: ",test_a)
print("Precision: ",test_p)
print("Recall: ",test_r)
print("F1 score: ",test_f)
print("Confusion Matrix: ",test_c)

print("True negative: ",tn)
print("False negative: ",fn)
print("True positive: ",tp)
print("False negative: ",fp)

Accuracy:  81.3
Precision:  0.9013597603134362
Recall:  0.8492942453854506
F1 score:  0.8745527728085867
Confusion Matrix:  [[ 967  428]
 [ 694 3911]]
True negative:  967
False negative:  694
True positive:  3911
False negative:  428
